In [1]:
import pandas as pd
import numpy as np
import re
from time import time
import sys
import pickle
import os

from keras.engine.topology import Layer
from keras.layers import InputSpec
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Dropout, Flatten, Lambda
from keras.layers import Conv2D, MaxPool2D, MaxPooling2D

from keras import backend as K
from keras.utils import conv_utils, to_categorical
from keras.datasets import mnist

from mid_max_pooling_2d import *

import math
import random
random.seed(91)

#import os
#os.environ["CUDA_VISIBLE_DEVICES"] = ""

/home/syht/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
batch_size = 32
num_classes = 10
epochs = 1

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)[:1000,:,:,:]
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)[:100,:,:,:]
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = to_categorical(y_train[:1000], num_classes)
y_test = to_categorical(y_test[:100], num_classes)

x_train shape: (1000, 28, 28, 1)
1000 train samples
100 test samples


In [3]:
def get_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=input_shape))
    #model.add(StochasticPooling(pool_size=(2, 2), modify=False))
    model.add(MidMaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(16, (3, 3), activation='relu'))
    #model.add(StochasticPooling(pool_size=(2, 2), modify=False))
    model.add(MidMaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    model.summary()
    return model

In [4]:
model = get_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
mid_max_pooling2d_1 (MidMaxP (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 16)        4624      
_________________________________________________________________
mid_max_pooling2d_2 (MidMaxP (None, 6, 6, 16)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 6, 6, 16)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               73856     
__________

In [5]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=20,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 1000 samples, validate on 100 samples
Epoch 1/20
 576/1000 [================>.............] - ETA: 14s - loss: 2.2404 - acc: 0.1944 

KeyboardInterrupt: 